# Sitemap Assay

The start of a simple notebook that could hosted for peeopl to test out their sitemaps (and robots.txt) files with.

References:
* [AdvTools](https://advertools.readthedocs.io/en/master/advertools.sitemaps.html)
* [Sitemap viz](https://www.ayima.com/us/insights/analytics-and-cro/how-to-visualize-an-xml-sitemap-using-python.html)



## Notes

has json-ld :  https://researchdata.edu.au/imos-soop-underway-sep-2017/970828
Validation:  https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fimos-soop-underway-sep-2017%2F970828  


doesn't:  https://researchdata.edu.au/heupel-michelle/1709766

Validation   https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fheupel-michelle%2F1709766


Context as map:  https://www.w3.org/TR/json-ld/#context-definitions





In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import advertools as adv
import json
import requests
from pyld import jsonld
from bs4 import BeautifulSoup
import urllib.request
import logging
import traceback
import kglab
import pandas as pd


In [3]:
%%time 

# smurl = 'https://opencoredata.org/sitemap_0.xml'
# smurl = 'https://www.bco-dmo.org/sitemap.xml'
# smurl = 'https://opentopography.org/sitemap.xml'
# smurl = 'https://www2.earthref.org/MagIC/contributions.sitemap.xml'
# smurl = 'https://oceanscape.org/organisation-sitemap.xml'
# smurl = 'https://catalogue.cioosatlantic.ca/sitemap/sitemap-1.xml'
# smurl = 'https://obis.org/sitemap/sitemap_datasets.xml'
# smurl = 'https://infohub.eurocean.net/sitemap/vessels'
# smurl = 'https://raw.githubusercontent.com/iodepo/odis-arch/schema-dev-jm/code/notebooks/Exploration/data-pacificdatahub/sitemap.xml'
# smurl = 'https://catalogue.cioospacific.ca/sitemap/sitemap-1.xml'
# smurl = 'https://geonode.nodc.id/sitemap.xml'
# smurl = "https://lipdverse.org/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://pacificdata.org/data/sitemap/cookislands-data/sitemap.xml"
# smurl = 'https://geoconnex.us/sitemap/ref/states/states__0.xml'
# smurl = "https://oceanscape.org/organisation-sitemap.xml"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
# smurl = "https://geoconnex.us/sitemap.xml"
# smurl = "https://ioc-africa.org/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://raw.githubusercontent.com/BeBOP-OBON/odis-interface/main/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
smurl = "https://metadata.naturalsciences.be/geonetwork/srv/api/sitemap"

iow_sitemap = adv.sitemap_to_df(smurl) # load sitemap to dataframe via advertools
# iow_sitemap.info()
# iow_sitemap.head()

2023-03-31 08:32:54,403 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://catalogue.cioos.ca/sitemap/sitemap-1.xml


CPU times: user 30.2 ms, sys: 3.14 ms, total: 33.3 ms
Wall time: 574 ms


## Analyzing the URLs

We can quickly grab the unique URLs from the sitemap column and see how many unique sitemap.xml files we are working with

We can also dive into the URL structure for the resources a bit.

In [4]:
usm = iow_sitemap.sitemap.unique()
uloc = iow_sitemap["loc"].unique()
print("{} unique sitemap XML file(s) pointing to {} unique resource(s).".format(len(usm), len(uloc)))


1 unique sitemap XML file(s) pointing to 1516 unique resource(s).


In [7]:

from urllib.parse import urlparse

invalid_urls = []

for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            pass
        else:
            invalid_urls.append(url)
    except ValueError:
        print(f"{url} is an exception URL")
        invalid_urls.append(url)


df = pd.DataFrame({'Invalid URLs': invalid_urls})


In [8]:
df.head()

,Invalid URLs


In [9]:
from urllib.parse import urlparse

valid_urls = []


for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            valid_urls.append(url)
        else:
            print(f"Invalid URL: {url}")
    except ValueError:
        print(f"Invalid URL: {url}")


In [10]:

# Break down all the URL into their path parts
urldf = adv.url_to_df(list(iow_sitemap['loc']))
# urldf = adv.url_to_df(list(uloc))

urldf.head()

,url,scheme,netloc,path,query,fragment,dir_1,dir_2,last_dir
0,https://catalogue.cioos.ca/dataset/5c5ff1a4-ba...,https,catalogue.cioos.ca,/dataset/5c5ff1a4-baeb-45ad-99d0-df1e3a5b386e,NaN,NaN,dataset,5c5ff1a4-baeb-45ad-99d0-df1e3a5b386e,5c5ff1a4-baeb-45ad-99d0-df1e3a5b386e
1,https://catalogue.cioos.ca/dataset/a8cfb216-ee...,https,catalogue.cioos.ca,/dataset/a8cfb216-eef2-4b13-9c3f-6b3aa91509e3,NaN,NaN,dataset,a8cfb216-eef2-4b13-9c3f-6b3aa91509e3,a8cfb216-eef2-4b13-9c3f-6b3aa91509e3
2,https://catalogue.cioos.ca/dataset/3d0c057b-ab...,https,catalogue.cioos.ca,/dataset/3d0c057b-ab39-4a7b-8809-4367d5028c11,NaN,NaN,dataset,3d0c057b-ab39-4a7b-8809-4367d5028c11,3d0c057b-ab39-4a7b-8809-4367d5028c11
3,https://catalogue.cioos.ca/dataset/c3817846-9a...,https,catalogue.cioos.ca,/dataset/c3817846-9ae7-4bbb-921f-152acb74f5fe,NaN,NaN,dataset,c3817846-9ae7-4bbb-921f-152acb74f5fe,c3817846-9ae7-4bbb-921f-152acb74f5fe
4,https://catalogue.cioos.ca/dataset/d2f02669-43...,https,catalogue.cioos.ca,/dataset/d2f02669-43a9-4aed-9646-8bc242522691,NaN,NaN,dataset,d2f02669-43a9-4aed-9646-8bc242522691,d2f02669-43a9-4aed-9646-8bc242522691


## Sample and test sitemap entries

In [11]:
# sample the previously generated url data frame
sample_size = 5
# sample_df = urldf.groupby("dir_1").sample(n=sample_size, random_state=1, replace=True)
sample_df = urldf.sample(n=sample_size, random_state=1, replace=True)

In [12]:
sample_df.head(5)

,url,scheme,netloc,path,query,fragment,dir_1,dir_2,last_dir
1061,https://catalogue.cioos.ca/dataset/5e2a843a-f4...,https,catalogue.cioos.ca,/dataset/5e2a843a-f4c5-48c9-adfd-99e299a34b10,NaN,NaN,dataset,5e2a843a-f4c5-48c9-adfd-99e299a34b10,5e2a843a-f4c5-48c9-adfd-99e299a34b10
235,https://catalogue.cioos.ca/dataset/ca-cioos_62...,https,catalogue.cioos.ca,/dataset/ca-cioos_62dc9c32-db61-40de-b5bb-8daf...,NaN,NaN,dataset,ca-cioos_62dc9c32-db61-40de-b5bb-8daf0307b1c51,ca-cioos_62dc9c32-db61-40de-b5bb-8daf0307b1c51
1096,https://catalogue.cioos.ca/dataset/1be308a2-e9...,https,catalogue.cioos.ca,/dataset/1be308a2-e98c-4af5-961d-194efb06eecc,NaN,NaN,dataset,1be308a2-e98c-4af5-961d-194efb06eecc,1be308a2-e98c-4af5-961d-194efb06eecc
905,https://catalogue.cioos.ca/dataset/741b95fa-8a...,https,catalogue.cioos.ca,/dataset/741b95fa-8a6c-40fd-bbf0-77c7cc59d597,NaN,NaN,dataset,741b95fa-8a6c-40fd-bbf0-77c7cc59d597,741b95fa-8a6c-40fd-bbf0-77c7cc59d597
715,https://catalogue.cioos.ca/dataset/b284dca4-ed...,https,catalogue.cioos.ca,/dataset/b284dca4-edcf-4051-b87e-81b61924a77c,NaN,NaN,dataset,b284dca4-edcf-4051-b87e-81b61924a77c,b284dca4-edcf-4051-b87e-81b61924a77c


### See if the URLs resolve

In [13]:
import urllib.request
import requests

ul = sample_df["url"]

for item in ul:
    # user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    # headers={'User-Agent':user_agent,}

    headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                           'AppleWebKit/537.11 (KHTML, like Gecko) '
                           'Chrome/23.0.1271.64 Safari/537.11',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}

    try:
        # x = requests.get(item)
        # code = x.status_code
        request=urllib.request.Request(url=item, headers=headers) #The assembled request
        with urllib.request.urlopen(request) as response:
            info = response.info()
            dtype = info.get_content_type()    # -> text/html
         # headers = x.headers()
        # print("URL: {} \ninfo : {}\n --".format(item, info))
        print("URL: {} ".format(item))
    except Exception as e:
        # code = x.status_code
        # dtype = info.get_content_type()

        print("Exception on: {} \nerrors : {}\n --".format(item, str(e)))


URL: https://catalogue.cioos.ca/dataset/5e2a843a-f4c5-48c9-adfd-99e299a34b10 
URL: https://catalogue.cioos.ca/dataset/ca-cioos_62dc9c32-db61-40de-b5bb-8daf0307b1c51 
URL: https://catalogue.cioos.ca/dataset/1be308a2-e98c-4af5-961d-194efb06eecc 
URL: https://catalogue.cioos.ca/dataset/741b95fa-8a6c-40fd-bbf0-77c7cc59d597 
URL: https://catalogue.cioos.ca/dataset/b284dca4-edcf-4051-b87e-81b61924a77c 


### See if they have JSON-LD (static check only, no dynamically loaded JSON-LD yet)

In [15]:
ul = sample_df["url"]

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    print(item)
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()
    
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        # print("JSON byte size: {} ".format(len(p)))
        print(p)
        print("JSON byte size: {} ".format(len(p.contents[0])))
    except Exception as e:
        logging.error(traceback.format_exc())

https://researchdata.edu.au/imos-soop-underway-sep-2017/970828
<script type="application/ld+json">{"@context":"https:\/\/schema.org\/","@type":"Dataset","creator":[{"@type":"Role","roleName":"Contributor","creator":{"@type":"Person","name":"Australian Institute of Marine Science (AIMS)"}},{"@type":"Role","roleName":"Contributor","creator":{"@type":"Person","name":"Integrated Marine Observing System (IMOS)"}}],"datePublished":"2023","sourceOrganization":{"@type":"Organization","name":"Integrated Marine Observing System"},"keywords":" oceans, Practical salinity of the water body, Temperature of the water body, Fluorescence of the water body, Turbidity of the water body","license":["CC-BY","http:\/\/creativecommons.org\/licenses\/by\/3.0\/au\/","Creative Commons Attribution 3.0 Australia License","CC-BY","http:\/\/creativecommons.org\/licenses\/by\/3.0\/au"],"publisher":{"@type":"Organization","name":"Australian Institute of Marine Science (AIMS)"},"spatialCoverage":[{"@type":"Place","geo

2023-03-15 14:12:22,596 | ERROR | 1914706094.py:25 | <module> | Traceback (most recent call last):
  File "/tmp/ipykernel_3460541/1914706094.py", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'



None
https://researchdata.edu.au/wienecke-barbara/475684


2023-03-15 14:12:25,436 | ERROR | 1914706094.py:25 | <module> | Traceback (most recent call last):
  File "/tmp/ipykernel_3460541/1914706094.py", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'



None
https://researchdata.edu.au/imos-soop-underway-jan-2019/1360836
<script type="application/ld+json">{"@context":"https:\/\/schema.org\/","@type":"Dataset","creator":[{"@type":"Role","roleName":"Contributor","creator":{"@type":"Person","name":"Australian Institute of Marine Science (AIMS)"}},{"@type":"Role","roleName":"Contributor","creator":{"@type":"Person","name":"Integrated Marine Observing System (IMOS)"}}],"datePublished":"2023","sourceOrganization":{"@type":"Organization","name":"Integrated Marine Observing System"},"keywords":" oceans, Practical salinity of the water body, Temperature of the water body, Fluorescence of the water body, Turbidity of the water body","license":["CC-BY","http:\/\/creativecommons.org\/licenses\/by\/3.0\/au\/","Creative Commons Attribution 3.0 Australia License","CC-BY","http:\/\/creativecommons.org\/licenses\/by\/3.0\/au"],"publisher":{"@type":"Organization","name":"Australian Institute of Marine Science (AIMS)"},"spatialCoverage":[{"@type":"Place

### Check JSON-LD structure (static check only, no dynamically loaded JSON-LD yet)

In [14]:
ul = sample_df["url"]

myframe =  {
    "@context":{"@vocab": "http://schema.org/"},
    "@type": "Dataset",
}

context =  { "@vocab": "http://schema.org/" }

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        jld = json.loads(p.contents[0])
        # jld = json.loads(p)

        # print(str(jld))
        # compacted = jsonld.compact(str(jld), context)
        # print(len(json.dumps(compacted, indent=2)))
    except Exception as e:
        print("Exception")
        logging.error(traceback.format_exc())

2023-03-15 14:10:55,496 | ERROR | 2320886990.py:34 | <module> | Traceback (most recent call last):
  File "/tmp/ipykernel_3460541/2320886990.py", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'



Exception


2023-03-15 14:10:58,107 | ERROR | 2320886990.py:34 | <module> | Traceback (most recent call last):
  File "/tmp/ipykernel_3460541/2320886990.py", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'



Exception


## Load to Graph"

Looad a sample set of triples into RDF lib and run a sample SPARQL query on them.

### Note
This is the same loop as above but now we will load into a KG graph

In [10]:
ul = sample_df["url"]

# Test loading into a graph
namespaces = {
    "schema":  "http://schema.org/",
    "schemaold":  "http://schema.org/",
    "shacl":   "http://www.w3.org/ns/shacl#" ,
}

kg = kglab.KnowledgeGraph(
    name = "Schema.org shacl eval datagraph",
    base_uri = "https://example.org/id/",
    namespaces = namespaces,
)

for item in ul:
    html = urllib.request.urlopen(item).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        print("JSON byte size: {} ".format(len(p.contents[0])))
        kg.load_rdf_text(data=p.contents[0], format="json-ld")
        print(p.contents[0])
    except Exception as e:
        logging.error(traceback.format_exc())

HTTPError: HTTP Error 403: Forbidden

In [ ]:
sparql = """
PREFIX schema: <http://schema.org/>
SELECT ?s ?name ?description ?type
  WHERE {
    ?s a ?type .    
    ?s schema:name ?name .
    ?s schema:description ?description.
  }
"""

#  schema:Dataset 
# ?s schema:name ?name .
#     ?s schema:description ?description.

df = kg.query_as_df(sparql).to_pandas()

df.head()